In [1]:
import os                                                                          
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"                                       
os.environ["CUDA_VISIBLE_DEVICES"]="0"  
from tensorflow.contrib.slim.nets import resnet_v2, resnet_utils
import tensorflow as tf
from tensorflow.contrib import layers as layers_lib
from tensorflow.python.ops import variable_scope
from tensorflow.contrib.layers.python.layers import utils
from tensorflow.contrib import slim
from tensorflow.nn import ctc_loss, conv2d
import numpy as np
resnet_v2_block = resnet_v2.resnet_v2_block
resnet_v2 = resnet_v2.resnet_v2

import matplotlib.pyplot as plt

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
BATCH_SIZE = 32
RESNET_STRIDE = 1
IMAGE_HEIGHT = 32

In [3]:
def resnet_v2_26_base(inputs,
                 num_classes=None,
                 is_training=True, # True - due to update batchnorm layers
                 global_pool=False,
                 output_stride=1, # effective stride 
                 reuse=None,
                 include_root_block=False, #first conv layer. Removed due to max pool supression. We need large receprive field
                 scope='resnet_v2_26'):
  
    """
    Tensorflow resnet_v2 use only bottleneck blocks (consist of 3 layers).
    Thus, this resnet layer model consist of 26 layers.
    I put stride = 2 on each block due to increase receptive field.

    """
    blocks = [
      resnet_v2_block('block1', base_depth=64, num_units=2, stride=2),
      resnet_v2_block('block2', base_depth=128, num_units=2, stride=2),
      resnet_v2_block('block3', base_depth=256, num_units=2, stride=2),
      resnet_v2_block('block4', base_depth=512, num_units=2, stride=2),
    ]
    return resnet_v2(
      inputs,
      blocks,
      num_classes,
      is_training,
      global_pool,
      output_stride,
      include_root_block,
      reuse=reuse,
      scope=scope)

def make_ocr_net(inputs, num_classes, is_training=False):
    '''
    Creates neural network graph.
    Image width halved and it's define timestamps width (feature sequence length) 
    No activation after output (no softmax), due to it's presence at ctc_loss() and beam_search().
    After resnet head features are resized to be [batch,1,width,channel], and after that goes 1x1 conv 
    to make anology of dense connaction for each timestamp.
    
    input: batch of images
    output: tensor of size [batch, time_stamps_width, num_classes]
    '''
    with tf.variable_scope('resnet_base', values=[inputs]) as sc:
        with slim.arg_scope([slim.conv2d],
                              activation_fn=None, normalizer_fn=None):
            net = resnet_utils.conv2d_same(inputs, 64, 7, stride=2, scope='conv1') #root conv for resnet
            #net = slim.max_pool2d(net, [3, 3], stride=2, scope='pool1') # due to enlarge of receptive field
            net = resnet_v2_26_base(net, output_stride=1, is_training = is_training)[0] # ouput is a tuple of last tensor and all tensors 
    with tf.variable_scope('class_head', values=[net]) as sc:
        net = tf.transpose(net, [0,3,1,2]) # next 4 lines due to column to channel reshape. [batch,c,h,w]
        _,c,h,_ = net.get_shape() # depth of input to conv op tensor should be static (defined)
        shape = tf.shape(net)
        net = tf.reshape(net, [shape[0], c*h, 1, shape[3]])
        net = tf.transpose(net,[0,2,3,1]) # back to [batch,h,w,c] = [batch,1,w,features*h]
        net = layers_lib.conv2d(net, num_classes, [1, 1], activation_fn=None) #CTC got softmax [batch,1,w,num_classes]
        net = tf.squeeze(net,1) #[batch,w,num_classes]
        return net

def ctc_loss_layer(sequence_labels, logits, sequence_length):
    """
    Build CTC Loss layer for training
    sequence_length is a list of siquences lengths, len(sequence_length) = batch_size.
    In our case sequences can not be different size due to it origin of images batch, 
    which should be of equal size (e.g. padded)
    """
    loss = tf.nn.ctc_loss( sequence_labels, 
                           logits, 
                           sequence_length,
                           time_major=False,  # [batch_size, max_time, num_classes] for logits
                           ignore_longer_outputs_than_inputs=True )
    total_loss = tf.reduce_mean( loss )
    return total_loss

def get_training(sequence_labels, net_logits, sequence_length, 
                   learning_rate=1e-4, decay_steps=2**16, decay_rate=0.9, decay_staircase=False, 
                   momentum=0.9):
    """
    Set up training ops
    https://github.com/weinman/cnn_lstm_ctc_ocr/blob/master/src/model_fn.py
    """
    with tf.name_scope( "train" ):
        net_logits_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)        
        loss = ctc_loss_layer(sequence_labels, net_logits, sequence_length) 
        # Update batch norm stats [http://stackoverflow.com/questions/43234667]
        extra_update_ops = tf.get_collection( tf.GraphKeys.UPDATE_OPS )
        with tf.control_dependencies( extra_update_ops ):
            # Calculate the learning rate given the parameters
            learning_rate_tensor = tf.train.exponential_decay(
                learning_rate,
                tf.train.get_global_step(),
                decay_steps,
                decay_rate,
                staircase=decay_staircase,
                name='learning_rate' )
            optimizer = tf.train.AdamOptimizer(
                learning_rate=learning_rate_tensor,
                beta1=momentum )
            train_op = tf.contrib.layers.optimize_loss(
                loss=loss,
                global_step=tf.train.get_global_step(),
                learning_rate=learning_rate_tensor, 
                optimizer=optimizer,
                variables=net_logits_vars)
            tf.summary.scalar('learning_rate', learning_rate_tensor )
    return train_op, loss, learning_rate_tensor

def get_prediction(output_net, seq_len, merge_repeated=False):
    '''
    predict by using beam search
    input: output_net - logits (without softmax) of net
           seq_len - length of predicted sequence 
    '''
    net = tf.transpose(output_net, [1, 0, 2]) #transpose to [time, batch, logits]
    decoded, prob = tf.nn.ctc_beam_search_decoder(net, seq_len, merge_repeated=merge_repeated)
    return decoded, prob

In [27]:
class OCRModel(object):
    def __init__(self, num_classes, input_image_batch, sequence_labels, is_training=True, learning_rate=1e-4,
                decay_steps=2**16, decay_rate=0.9, decay_staircase=False, momentum=0.9):
        self.num_classes = num_classes
        self.is_training = is_training
        self.learning_rate = learning_rate
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate 
        self.decay_staircase = decay_staircase
        self.momentum = momentum
        
        self.build(input_image_batch, sequence_labels)
        
    def build(self, input_image_batch, sequence_labels):
        
        self.input_image_batch = input_image_batch
        self.sequence_labels = sequence_labels
        self.feature_seq_length = tf.fill([tf.shape(self.input_image_batch)[0]], tf.shape(self.input_image_batch)[2]//(2*RESNET_STRIDE)) #as we know effective stride
        
        net = make_ocr_net(self.input_image_batch, self.num_classes, is_training=self.is_training)
        self.net = net
        self.train_op, self.loss, self.learning_rate_tensor = get_training(self.sequence_labels, net, self.feature_seq_length,
                                                self.learning_rate, self.decay_steps, 
                                                self.decay_rate, self.decay_staircase, self.momentum)
        self.prediction = get_prediction(net, self.feature_seq_length, merge_repeated=False) # tuple(decoded, prob). decoded - list of top paths. I use top1
        self.lev_dist = tf.reduce_mean(tf.edit_distance(tf.cast(self.prediction[0][0], tf.int32), self.sequence_labels))
        tf.summary.scalar('CTC loss', self.loss)
        tf.summary.scalar('Levenshtein distance', self.lev_dist)
        tf.summary.scalar('Learning rate', self.learning_rate_tensor)
        self.merged_summary = tf.summary.merge_all()

In [28]:
import sys
sys.path.append('../../data_generator/')
from data_generator import data_generator
dg = data_generator(backgrounds_path='../../data_generator/backgrounds/', 
                                   fonts_path='../../data_generator/valid_fonts/',
                                   valid_charset_path='../../data_generator/valid_charset.txt')

In [32]:
def local_data_generator(*args):
    image, label = dg.get_image_and_label()
    image = image/255
    image = image - 0.5
    return image.astype(np.float32), label.astype(np.int32)
def set_shape_func(image, label):
    image.set_shape([IMAGE_HEIGHT, None, 1])
    return image, label

In [33]:
# all_chars = ' !"%&()*+,-./0123456789:=ABCDEFGHIJKLMNOPQRSTUVWXYZ\\_abcdefghijklmnopqrstuvwxyz|~ЁАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё№'
all_chars = list(set(dg.valid_charset))
char_to_indx = dict(zip(all_chars,range(len(all_chars))))
num_classes = len(all_chars)
# def string_to_label(string):
#     label = [char_to_indx[s] for s in string]
#     return label

# def batch_to_sparse(batch, dtype=np.int32): #batch of words
#     '''
#     function return sparce represantance of labels.
#     input: batch - batch of words (List of words)
#     output: indices - list of indexes [batch_num,time_stamp_num]
#             values - list of char indexes shape [batch]
#             shape - shape of dense batch represantation
#     '''
#     assert isinstance(batch, list) or isinstance(batch, np.ndarray), 'batch should be a list or numpy array of strings'
#     indices = [] #[batch_num,w]
#     values = [] # char indx
#     for batch_num, word in enumerate(batch):
#         assert isinstance(word,str), 'batch element should be a string'
#         word_as_indx = string_to_label(word)
#         indices.extend([(batch_num,char_num) for char_num, char in enumerate(word_as_indx)])
#         values.extend([char for char_num, char in enumerate(word_as_indx)])
#     indices = np.asarray(indices, dtype=dtype)
#     values = np.asarray(values, dtype=dtype)
#     shape = np.array([len(batch),indices.max(0)[1]+1], dtype=dtype)
#     return indices, values, shape
# def from_sparse_to_batch():
#     pass

In [34]:

graph = tf.Graph()
with graph.as_default():
#     dataset = tf.data.Dataset().from_generator(dg.get_image_and_label, output_types= (tf.float32, (tf.int64,tf.int32,tf.int64)),
#                                               output_shapes = (tf.TensorShape([None,32,None,1]), 
#                                                                           (tf.TensorShape([None,None]),
#                                                                            tf.TensorShape([None]),
#                                                                            tf.TensorShape([None]))))
    dataset = tf.data.Dataset().range(int(1e8))
#     dataset = dataset.map(local_data_generator, num_parallel_calls=4).prefetch(32)
#     dataset = dataset.map(set_shape_func, num_parallel_calls=4).prefetch(32)
    dataset = dataset.interleave(lambda x: tf.data.Dataset.from_tensors(local_data_generator()), cycle_length=32, block_length=32)
    dataset = dataset.batch(BATCH_SIZE).prefetch(32)
    data_iter = dataset.make_initializable_iterator()
    features, labels = data_iter.get_next()
    labels = tf.contrib.layers.dense_to_sparse(labels)
    tf.train.create_global_step()
    model = OCRModel(num_classes=num_classes+2, input_image_batch=features, 
                     sequence_labels=labels, is_training=True)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

INFO:tensorflow:Summary name CTC loss is illegal; using CTC_loss instead.


INFO:tensorflow:Summary name CTC loss is illegal; using CTC_loss instead.


INFO:tensorflow:Summary name Levenshtein distance is illegal; using Levenshtein_distance instead.


INFO:tensorflow:Summary name Levenshtein distance is illegal; using Levenshtein_distance instead.


INFO:tensorflow:Summary name Learning rate is illegal; using Learning_rate instead.


INFO:tensorflow:Summary name Learning rate is illegal; using Learning_rate instead.


In [ ]:
import time
tic = time.time()
LOG_DIR = 'log_temp_temp/'
with tf.Session(graph=graph) as sess:
    train_writer = tf.summary.FileWriter(LOG_DIR + 'train',
                                          sess.graph)
    test_writer = tf.summary.FileWriter(LOG_DIR + 'test')
    sess.run([init, data_iter.initializer])
#     try:
#         saver.restore(sess, "log/model.ckpt")
#     except:
#         print('cant restore')
    for num in range(int(10)):
        print(num)
        _, ms = sess.run([model.train_op, model.merged_summary])
        train_writer.add_summary(ms, num)
        if num%100 == 0:
            _, ms_ = sess.run([model.train_op, model.merged_summary])
            test_writer.add_summary(ms_, num)
            saver.save(sess, LOG_DIR + "model.ckpt")
print(time.time() - tic)

0
1


KeyboardInterrupt: 